In [128]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import numpy as np
import cmcrameri.cm as cmc
from matplotlib.colors import rgb2hex
import h5py
import datetime as dt
import xyzservices.providers as xyz

In [129]:
creepmeters = pd.read_excel('../../Data/Creepmeter_details_for_map.xlsx')
creepmeters.drop_duplicates('Creepmeter_abbrv',inplace=True)

networks = np.unique(creepmeters['Network'])
cmap=cmc.managua
colours  = cmap(np.linspace(0, 1, len(networks)+1))
df_all = pd.read_csv("../../Data/DATA_tidied/Picks/All_picks_23_APR_2025.csv",index_col=0)
df_all['ST'] = pd.to_datetime(df_all['ST'])
df_all.drop(df_all[df_all['ST']>dt.datetime(2024,1,1,0,0,0)].index,inplace=True)
df_all.drop(df_all[df_all['Displacement, mm']<0.02].index,inplace=True)

In [130]:
def start_end(path):
    with h5py.File(path, 'r') as f:
        # Loop through each key in the file
        starts = []
        ends = []
        for key in f.keys():
            if key not in ['Temperature','Temperature_1T','Temperature_5T','Daily_measurements','Manual_measurements','Orthogonal']:
                data = f[key]  # Access the dataset
                data.attrs
                data_keys = list(data.keys())
                time =  data[data_keys[1]][:]
                decoded_time = [byte_str.decode('utf-8') for byte_str in time]
                decoded_time = pd.to_datetime(decoded_time)
                starts.append(decoded_time[0])
                ends.append(decoded_time[-1])
    ST = np.min(starts)
    ET = np.max(ends)
    return ST, ET

def event_no(picks, abbrv):
    instrument_df = picks[picks['Creepmeter_abbrv'] == abbrv]
    no_events = len(instrument_df)
    return no_events

In [132]:
m = folium.Map(location=[0, 0], zoom_start=3, min_zoom=3,)
marker_cluster = MarkerCluster().add_to(m)
for i in range(len(networks)):
    creepmeters_network = creepmeters.drop(creepmeters[creepmeters['Network']!=networks[i]].index)
    creepmeters_network.reset_index(inplace=True,drop=True)
    for j in range(len(creepmeters_network)):
        path = '../../Data/DATA_tidied/HDF5/{k}.h5'.format(k=creepmeters_network['Creepmeter_abbrv'].iloc[j])
        start,end = start_end(path)
        no_events = event_no(df_all,creepmeters_network['Creepmeter_abbrv'].iloc[j])
        
        popup_content = f"""
        <b>Creepmeter name:</b> {creepmeters_network['Creepmeter_full_name'].iloc[j]}<br>
        <b>Creepmeter abbreviation:</b> {creepmeters_network['Creepmeter_abbrv'].iloc[j]} 
        <b>Obliquity:</b> {creepmeters_network['Obliquity'].iloc[j]} 
        <b>Start Time:</b> {start} 
        <b>End Time:</b> {end} 
        <b>Number of events:</b> {no_events} 
 
    """
        popup = folium.Popup(popup_content, max_width=400)
        folium.Marker(
            location=[creepmeters_network['Latitude'].iloc[j],creepmeters_network['Longitude'].iloc[j]],
            popup=popup,
            icon=folium.Icon(color='gray', icon_color=rgb2hex(colours[i]), icon="caret-down", prefix='fa'),
        ).add_to(marker_cluster)

In [118]:
m

In [131]:
m = folium.Map(location=[39.88, -105.08], zoom_start=10, tiles=xyz.Stamen.Terrain())
m